In [1]:
from IPython.core.display import display, HTML; display(HTML("<style>.container{width:90% !important;}</style>"))
%autosave 30

Autosaving every 30 seconds


# Grzegorz Nowak - Recency, Frequency, Engagement (RFE) analysis of the wines' qualities.

## 0. Description of the problem.

### The goal is to classify wines' types (red / white) using 'Wine Quality' data-set (https://archive.ics.uci.edu/ml/datasets/Wine+Quality) available in the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets.php). In fact, these are two data-sets related to red and white variants of the Portuguese "Vinho Verde" wine. They are directly available under this link: https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/.

### The data-sets and their possible modelling are described in details in the paper of P. Cortez, A. Cerdeira, F. Almeida, T. Matos & J. Reis. "Modeling wine preferences by data mining from physicochemical properties", published in "Decision Support Systems", Elsevier, Volume 47, Issue 4, Pages 547-553 in November 2009 (https://www.sciencedirect.com/science/article/abs/pii/S0167923609001377?via%3Dihub).

### Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (i.e. there is no data about grape types, wine brand, wine selling price, etc.).

### Above-mentioned input variables that base on the physicochemical tests are included in the first 11 columns of the data-sets:
#### 00 - fixed acidity
#### 01 - volatile acidity
#### 02 - citric acid
#### 03 - residual sugar
#### 04 - chlorides
#### 05 - free sulfur dioxide
#### 06 - total sulfur dioxide
#### 07 - density
#### 08 - pH
#### 09 - sulphates
#### 10 - alcohol
### The only one utput variable that base on the sensory data is given in the last, 12th column of the data-sets:
#### 11 - quality (score between 0 and 10)

### The classes are ordered and not balanced. It is not obvious if all input variables are relevant. Outlier detection algorithms could be used to detect the few excellent or poor wines.

### These data-sets can be viewed as classification or regression tasks. But in fact, it may be approached as a problem of risk assessment that is very common in the financial sector.

## 1. Required libraries.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, recall_score
from sklearn.feature_selection import RFE
from scipy.stats import randint
from sklearn import tree
#import graphviz

## 2. Input data.

### 2.1. Loading data.

In [3]:
wines_w = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep = ';')
wines_r = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep = ';')
#wines_w = pd.read_csv('input/winequality-white.csv', sep = ';')
#wines_r = pd.read_csv('input/winequality-red.csv', sep = ';')

### 2.2. Summarizing data-sets.

#### 2.2.1. Data-set "wines_w".

In [4]:
wines_w.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
9,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


In [5]:
print(f'"wines_w" data-set contains {wines_w.shape[1]} columns and {wines_w.shape[0]} rows.')
#wines_w.shape

"wines_w" data-set contains 12 columns and 4898 rows.


In [6]:
print(f'"wines_w" data-set contains following columns: {list(wines_w.columns)}.')
#wines_w.columns

"wines_w" data-set contains following columns: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality'].


In [7]:
print(f'"wines_w" data-set contains following types of data: {wines_w.columns.to_series().groupby(wines_w.dtypes).groups}.')

"wines_w" data-set contains following types of data: {int64: ['quality'], float64: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']}.


In [8]:
print(f'General information about "wines_w" data-set: {wines_w.info()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB
General information about "wines_w" data-set: None


#### 2.2.1. Data-set "wines_r".

In [9]:
wines_r.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


In [10]:
print(f'"wines_r" data-set contains {wines_r.shape[1]} columns and {wines_r.shape[0]} rows.')
#wines_w.shape

"wines_r" data-set contains 12 columns and 1599 rows.


In [11]:
print(f'"wines_r" data-set contains following columns: {list(wines_r.columns)}.')
#wines_w.columns

"wines_r" data-set contains following columns: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality'].


In [12]:
print(f'"wines_r" data-set contains following types of data: {wines_r.columns.to_series().groupby(wines_w.dtypes).groups}.')

"wines_r" data-set contains following types of data: {int64: ['quality'], float64: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']}.


In [13]:
print(f'General information about "wines_r" data-set: {wines_r.info()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
General information about "wines_r" data-set: None


***
#### Conclusions. Both "wines_w" and "wines_r" data-sets contains the same 12 columns. Therefore it is possible to merge them. But before merging them one can check for any possible missing values and duplicated raws.
***

### 2.3. Adding an additional categorical column with identification of red wines by 1 and white wines by 0.

In [14]:
wines_w['red_wine'] = 0
wines_r['red_wine'] = 1

In [15]:
wines_w.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6,0
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
9,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6,0


In [16]:
wines_r.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,1
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,1
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,1


### 2.4. Sanity cross-check for any missing values in one of the columns in both data-sets. The info() method used above already provided an information about the number of non-null values in each column of both data-sets, i.e. 4898 and 1599 for "wines_w" and "wines_r", respectively. Below analysis allows to check this directly.

#### 2.4.1. Data-set "wines_w".

In [17]:
print(f'"wines_w" data-set contains {wines_w.shape[0]} rows.')
print(f'Are there any missing values in one of the columns?')
print(f'Answer: {wines_w.isnull().values.any()}.')
print(f'Number of columns with missing values: {wines_w.isnull().values.any().sum()}.')

"wines_w" data-set contains 4898 rows.
Are there any missing values in one of the columns?
Answer: False.
Number of columns with missing values: 0.


In [18]:
for column in wines_w.columns:
    print('--------------------------------------------------------------------------------')
    print(column)
    print(wines_w[column].isnull().value_counts())

--------------------------------------------------------------------------------
fixed acidity
False    4898
Name: fixed acidity, dtype: int64
--------------------------------------------------------------------------------
volatile acidity
False    4898
Name: volatile acidity, dtype: int64
--------------------------------------------------------------------------------
citric acid
False    4898
Name: citric acid, dtype: int64
--------------------------------------------------------------------------------
residual sugar
False    4898
Name: residual sugar, dtype: int64
--------------------------------------------------------------------------------
chlorides
False    4898
Name: chlorides, dtype: int64
--------------------------------------------------------------------------------
free sulfur dioxide
False    4898
Name: free sulfur dioxide, dtype: int64
--------------------------------------------------------------------------------
total sulfur dioxide
False    4898
Name: total sulfur

#### 2.4.2. Data-set "wines_r".

In [19]:
print(f'"wines_r" data-set contains {wines_r.shape[0]} rows.')
print(f'Are there any missing values in one of the columns?')
print(f'Answer: {wines_r.isnull().values.any()}.')
print(f'Number of columns with missing values: {wines_r.isnull().values.any().sum()}.')

"wines_r" data-set contains 1599 rows.
Are there any missing values in one of the columns?
Answer: False.
Number of columns with missing values: 0.


In [20]:
for column in wines_r.columns:
    print('--------------------------------------------------------------------------------')
    print(column)
    print(wines_r[column].isnull().value_counts())

--------------------------------------------------------------------------------
fixed acidity
False    1599
Name: fixed acidity, dtype: int64
--------------------------------------------------------------------------------
volatile acidity
False    1599
Name: volatile acidity, dtype: int64
--------------------------------------------------------------------------------
citric acid
False    1599
Name: citric acid, dtype: int64
--------------------------------------------------------------------------------
residual sugar
False    1599
Name: residual sugar, dtype: int64
--------------------------------------------------------------------------------
chlorides
False    1599
Name: chlorides, dtype: int64
--------------------------------------------------------------------------------
free sulfur dioxide
False    1599
Name: free sulfur dioxide, dtype: int64
--------------------------------------------------------------------------------
total sulfur dioxide
False    1599
Name: total sulfur

***
#### Conclusions. There are any missing values in both data-sets.
***

### 2.5. Checking for duplicated raws and dropping them.

#### 2.5.1. Data-set "wines_w".

##### Number of duplicated rows.

In [21]:
wines_w_duplicated_sum = sum(wines_w.duplicated(subset=None, keep='first'))
print(f'Number of duplicated values in data-set "wines_w" (wines_w_duplicated_sum) is equal to {wines_w_duplicated_sum}.')

Number of duplicated values in data-set "wines_w" (wines_w_duplicated_sum) is equal to 937.


##### Quick visualization of duplicated rows.

In [22]:
wines_w_duplicated = wines_w[wines_w.duplicated(keep=False)].sort_values('fixed acidity').sort_values('volatile acidity')
wines_w_duplicated.head(10)##### Number of duplicated rows.

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
1480,7.2,0.080,0.49,1.3,0.050,18.0,148.0,0.99450,3.46,0.44,10.2,6,0
1475,7.2,0.080,0.49,1.3,0.050,18.0,148.0,0.99450,3.46,0.44,10.2,6,0
973,6.5,0.080,0.33,1.9,0.028,23.0,93.0,0.99100,3.34,0.70,12.0,7,0
968,6.5,0.080,0.33,1.9,0.028,23.0,93.0,0.99100,3.34,0.70,12.0,7,0
4801,6.4,0.105,0.29,1.1,0.035,44.0,140.0,0.99142,3.17,0.55,10.7,7,0
4795,6.4,0.105,0.29,1.1,0.035,44.0,140.0,0.99142,3.17,0.55,10.7,7,0
4796,6.4,0.105,0.29,1.1,0.035,44.0,140.0,0.99142,3.17,0.55,10.7,7,0
4252,6.7,0.110,0.26,14.8,0.053,44.0,95.0,0.99676,3.20,0.35,9.8,6,0
4250,6.7,0.110,0.26,14.8,0.053,44.0,95.0,0.99676,3.20,0.35,9.8,6,0
769,6.7,0.110,0.34,8.8,0.043,41.0,113.0,0.99620,3.42,0.40,9.3,7,0


##### Dropping duplicated rows.

In [23]:
wines_w_cleaned = wines_w.drop_duplicates()
wines_w_cleaned.reset_index()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
0,0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,0
1,1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,0
2,2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,0
3,3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,0
4,6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.99490,3.18,0.47,9.6,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3956,4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
3957,4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
3958,4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
3959,4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


#### 2.5.2. Data-set "wines_r".

##### Number of duplicated rows.

In [24]:
wines_r_duplicated_sum = sum(wines_r.duplicated(subset=None, keep='first'))
print(f'Number of duplicated values in data-set "wines_r" (wines_r_duplicated_sum) is equal to {wines_r_duplicated_sum}.')

Number of duplicated values in data-set "wines_r" (wines_r_duplicated_sum) is equal to 240.


##### Quick visualization of duplicated rows.

In [25]:
wines_r_duplicated = (wines_r[wines_r.duplicated(keep=False)].sort_values('fixed acidity')).sort_values('volatile acidity')
wines_r_duplicated.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
948,8.9,0.12,0.45,1.8,0.075,10.0,21.0,0.99552,3.41,0.76,11.9,7,1
949,8.9,0.12,0.45,1.8,0.075,10.0,21.0,0.99552,3.41,0.76,11.9,7,1
950,8.9,0.12,0.45,1.8,0.075,10.0,21.0,0.99552,3.41,0.76,11.9,7,1
271,11.5,0.18,0.51,4.0,0.104,4.0,23.0,0.99960,3.28,0.97,10.1,6,1
277,11.5,0.18,0.51,4.0,0.104,4.0,23.0,0.99960,3.28,0.97,10.1,6,1
1018,8.0,0.18,0.37,0.9,0.049,36.0,109.0,0.99007,2.89,0.44,12.7,6,1
1017,8.0,0.18,0.37,0.9,0.049,36.0,109.0,0.99007,2.89,0.44,12.7,6,1
269,11.5,0.18,0.51,4.0,0.104,4.0,23.0,0.99960,3.28,0.97,10.1,6,1
243,15.0,0.21,0.44,2.2,0.075,10.0,24.0,1.00005,3.07,0.84,9.2,7,1
244,15.0,0.21,0.44,2.2,0.075,10.0,24.0,1.00005,3.07,0.84,9.2,7,1


##### Dropping duplicated rows.

In [26]:
wines_r_cleaned = wines_r.drop_duplicates()
wines_r_cleaned.reset_index()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
0,0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,1593,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1
1355,1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1
1356,1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1
1357,1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,1


***
#### Conclusions. We removed all duplicated rows from both data-sets (937 from "wines_w" data-set and "wines_r" data-set and 240 from "wines_r" one). We also reset the indices in both data-sets.
***

### 2.5. Merging both data-sets into one.

In [27]:
wines = pd.concat([wines_w_cleaned, wines_r_cleaned], axis=0)

In [28]:
print(f'"wines" data-set contains {wines.shape[1]} columns and {wines.shape[0]} rows.')
#wines_w.shape

"wines" data-set contains 13 columns and 5320 rows.


In [29]:
wines.head(7)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
0,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
6,6.2,0.32,0.16,7.00,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6,0
9,8.1,0.22,0.43,1.50,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6,0
10,8.1,0.27,0.41,1.45,0.033,11.0,63.0,0.9908,2.99,0.56,12.0,5,0


In [30]:
wines.tail(7)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
1591,5.4,0.740,0.09,1.7,0.089,16.0,26.0,0.99402,3.67,0.56,11.6,6,1
1592,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1
1593,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,1
1598,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6,1


## 3. Basic statistical description of the data.

### 3.1. Categorical data.

In [31]:
wines.red_wine.value_counts(normalize = True)

0    0.744549
1    0.255451
Name: red_wine, dtype: float64

***
#### Conclusions. The merged data-set contains 74.4% of white wines and 25.6% of red wines. It is not very balanced data-set.
***

### 3.2. Numerical data.

In [32]:
wines.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
count,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000
mean,7.215179,0.344130,0.318494,5.048477,0.056690,30.036654,114.109023,0.994535,3.224664,0.533357,10.549241,5.795677,0.255451
std,1.319671,0.168248,0.147157,4.500180,0.036863,17.805045,56.774223,0.002966,0.160379,0.149743,1.185933,0.879772,0.436155
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.400000,0.230000,0.240000,1.800000,0.038000,16.000000,74.000000,0.992200,3.110000,0.430000,9.500000,5.000000,0.000000
50%,7.000000,0.300000,0.310000,2.700000,0.047000,28.000000,116.000000,0.994650,3.210000,0.510000,10.400000,6.000000,0.000000
75%,7.700000,0.410000,0.400000,7.500000,0.066000,41.000000,153.250000,0.996770,3.330000,0.600000,11.400000,6.000000,1.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000


In [33]:
wines.agg(['mean', 'median'])

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
mean,7.215179,0.34413,0.318494,5.048477,0.05669,30.036654,114.109023,0.994535,3.224664,0.533357,10.549241,5.795677,0.255451
median,7.000000,0.30000,0.310000,2.700000,0.04700,28.000000,116.000000,0.994650,3.210000,0.510000,10.400000,6.000000,0.000000
